In [5]:
import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

sns.set_style("darkgrid")

import matplotlib.ticker

# Number of ticks for yaxis
nticks = 5

In [6]:
dfs = []
for fn in snakemake.input["results"]:
    df = pd.read_csv(fn, delimiter=";")
    df["csp"] = "CSP" if "_with_csp" in fn else "no CSP"

    dfs.append(df)

df = pd.concat(dfs)

# Ensure correct sort order for all plots by specified list
df["scenario"] = pd.Categorical(
    df["scenario"],
    ["unbuffered", "daily", "weekly", "biweekly", "monthly", "quaterly", "annually"],
)

In [7]:
battery = df.query(
    "category == 'installed capacity' and subcategory == 'battery storage (exp)'"
)
battery["value"] /= 1e3
hvdc = df.query(
    "category == 'capacity factor' and subcategory == 'HVDC inverter pair (exp)'"
)

tes = df[
    (df["category"] == "installed capacity") & (df["subcategory"].str.contains("TES"))
]
tes = tes.groupby(["scenario", "esc", "exporter", "csp"])["value"].sum().reset_index()
tes["value"] /= 1e3

h2store = df.query(
    "category == 'installed capacity' and subcategory == 'hydrogen storage tank incl. compressor (exp)'"
)
h2store["value"] /= 1e3
h2pipeline = df.query(
    "category == 'capacity factor' and subcategory == 'H2 (g) fill compressor station (exp)'"
)

In [8]:
esc = "hvdc-to-elec"

for exporter in ["MA", "SA", "TN"]:
    csp = "no CSP"

    fig, axes = plt.subplots(ncols=1, nrows=2, sharex=True, figsize=(6, 6))
    axl = axes[0]
    axr = axl.twinx()

    battery.query("esc == @esc and exporter == @exporter and csp == @csp").plot(
        x="scenario", y="value", marker="<", ax=axl, label="battery", legend=False
    )
    hvdc.query("esc == @esc and exporter == @exporter and csp == @csp").plot(
        x="scenario",
        y="value",
        marker=">",
        ax=axr,
        ls="dashed",
        label="HVDC",
        legend=False,
    )

    # add all lines into one legend
    # ask matplotlib for the plotted objects and their labels
    lines, labels = axl.get_legend_handles_labels()
    lines2, labels2 = axr.get_legend_handles_labels()
    axl.legend(
        lines + lines2,
        labels + labels2,
        loc="center",
        bbox_to_anchor=(0.5, 1.25),
        title=f"{csp}",
        ncol=2,
    )

    axl.set_ylabel("battery capacity [GWh]")
    axr.set_ylabel("HVDC load factor [p.u.]")
    axl.set_xlabel("demand flexibility")

    axr.set_ylim(0, 1)
    axl.set_ylim(0, 68)  # battery["value"].max()*1.05)

    axl.yaxis.set_major_locator(matplotlib.ticker.LinearLocator(nticks))
    axr.yaxis.set_major_locator(matplotlib.ticker.LinearLocator(nticks))

    csp = "CSP"

    axl = axl = axes[1]
    axr = axl.twinx()

    tes.query("esc == @esc and exporter == @exporter and csp == @csp").plot(
        x="scenario", y="value", marker="<", ax=axl, label="TES", legend=False
    )
    battery.query("esc == @esc and exporter == @exporter and csp == @csp").plot(
        x="scenario", y="value", marker="<", ax=axl, label="battery", legend=False
    )
    hvdc.query("esc == @esc and exporter == @exporter and csp == @csp").plot(
        x="scenario",
        y="value",
        marker=">",
        ax=axr,
        ls="dashed",
        label="HVDC",
        legend=False,
    )

    # add all lines into one legend
    # ask matplotlib for the plotted objects and their labels
    lines, labels = axl.get_legend_handles_labels()
    lines2, labels2 = axr.get_legend_handles_labels()
    axl.legend(
        lines + lines2,
        labels + labels2,
        loc="center",
        bbox_to_anchor=(0.5, 1.25),
        title=f"{csp}",
        ncol=2,
    )

    axl.set_ylabel("TES capacity [GWh]")
    axr.set_ylabel("HVDC load factor [p.u.]")
    axl.set_xlabel("demand flexibility")

    axr.set_ylim(0, 1)
    axl.set_ylim(0, 640)  # tes["value"].max()*1.05)

    axl.yaxis.set_major_locator(matplotlib.ticker.LinearLocator(nticks))
    axr.yaxis.set_major_locator(matplotlib.ticker.LinearLocator(nticks))

    fig.suptitle(f"{exporter}: {esc}")
    fig.tight_layout()

    break

    fig.savefig(
        [fn for fn in snakemake.output["figures"] if exporter in fn and esc in fn][0],
        dpi=300,
    )

In [ ]:
esc = "pipeline-h2-to-elec"

for exporter in ["MA", "SA", "TN"]:
    csp = "no CSP"

    fig, axes = plt.subplots(ncols=1, nrows=2, sharex=True, figsize=(6, 6))
    axl = axes[0]
    axr = axl.twinx()

    h2store.query("esc == @esc and exporter == @exporter and csp == @csp").plot(
        x="scenario", y="value", marker="<", ax=axl, label="H2 store", legend=False
    )
    h2pipeline.query("esc == @esc and exporter == @exporter and csp == @csp").plot(
        x="scenario",
        y="value",
        marker=">",
        ax=axr,
        ls="dashed",
        label="H2 pipeline",
        legend=False,
    )

    # add all lines into one legend
    # ask matplotlib for the plotted objects and their labels
    lines, labels = axl.get_legend_handles_labels()
    lines2, labels2 = axr.get_legend_handles_labels()
    axl.legend(
        lines + lines2,
        labels + labels2,
        loc="center",
        bbox_to_anchor=(0.5, 1.25),
        title=f"{csp}",
        ncol=2,
    )

    axl.set_ylabel("H2 store capacity [GWh]")
    axr.set_ylabel("H2 pipeline load factor [p.u.]")
    axl.set_xlabel("demand flexibility")

    axr.set_ylim(0, 1.0)
    axl.set_ylim(0, 160)

    axl.yaxis.set_major_locator(matplotlib.ticker.LinearLocator(nticks))
    axr.yaxis.set_major_locator(matplotlib.ticker.LinearLocator(nticks))

    csp = "CSP"

    axl = axl = axes[1]
    axr = axl.twinx()

    tes.query("esc == @esc and exporter == @exporter and csp == @csp").plot(
        x="scenario", y="value", marker="<", ax=axl, label="TES", legend=False
    )
    h2pipeline.query("esc == @esc and exporter == @exporter and csp == @csp").plot(
        x="scenario",
        y="value",
        marker=">",
        ax=axr,
        ls="dashed",
        label="H2 pipeline",
        legend=False,
    )

    # add all lines into one legend
    # ask matplotlib for the plotted objects and their labels
    lines, labels = axl.get_legend_handles_labels()
    lines2, labels2 = axr.get_legend_handles_labels()
    axl.legend(
        lines + lines2,
        labels + labels2,
        loc="center",
        bbox_to_anchor=(0.5, 1.25),
        title=f"{csp}",
        ncol=2,
    )

    axl.set_ylabel("TES capacity [GWh]")
    axr.set_ylabel("H2 pipeline load factor [p.u.]")
    axl.set_xlabel("demand flexibility")

    axr.set_ylim(0, 1.0)
    axl.set_ylim(0, 640)

    axl.yaxis.set_major_locator(matplotlib.ticker.LinearLocator(nticks))
    axr.yaxis.set_major_locator(matplotlib.ticker.LinearLocator(nticks))

    fig.suptitle(f"{exporter}: {esc}")
    fig.tight_layout()

    fig.savefig(
        [fn for fn in snakemake.output["figures"] if exporter in fn and esc in fn][0],
        dpi=300,
    )